In [ ]:
import pathlib
import shutil
import os

import numpy as np
import pandas as pd

import dbfread

In [ ]:
original_dir = pathlib.Path(r'S:\DataExchange\SASH\iViewDB')
decoded_dir = pathlib.Path(r'S:\DataExchange\SASH\iViewDB_decoded')

decoded_dir.mkdir(exist_ok=True)

In [ ]:
table_copies_dir = decoded_dir.joinpath('table_copies')
table_copies_dir.mkdir(exist_ok=True)
to_be_indexed_dir = decoded_dir.joinpath('to_be_indexed')
to_be_indexed_dir.mkdir(exist_ok=True)
indexed_dir = decoded_dir.joinpath('indexed')
indexed_dir.mkdir(exist_ok=True)

In [ ]:
original_database_files = list(original_dir.glob('*.DBF')) + list(original_dir.glob('*.MDX')) + list(original_dir.glob('*.DBT'))
for filepath in original_database_files:
    new_path = table_copies_dir.joinpath(filepath.name)
    shutil.copy2(filepath, new_path)

In [ ]:
table_paths = {
    'patient': table_copies_dir.joinpath('PATIENT.DBF'),
    'treatment': table_copies_dir.joinpath('TRTMNT.DBF'),
    'port': table_copies_dir.joinpath('PORT.DBF'),
    'patient_image': table_copies_dir.joinpath('PATIMG.DBF'),
    'frame': table_copies_dir.joinpath('PORT_N.DBF'),
}

In [ ]:
table_records = {
    key: list(dbfread.DBF(table_path))
    for key, table_path in table_paths.items()
}

In [ ]:
images_to_be_indexed = list(to_be_indexed_dir.glob('*.jpg'))
file_image_ids = [
    int(path.stem.split('.')[0])
    for path in images_to_be_indexed
]

# image_ids

In [ ]:
table_ids = {
    key: [record['DBID'] for record in records]
    for key, records in table_records.items()
}
# table_ids

In [ ]:
pimg_ids = [record['PIMG_DBID'] for record in table_records['frame']]
pimg_id_unique, pimg_id_first_index, pimg_id_counts = np.unique(pimg_ids, return_index=True, return_counts=True)

pimg_id_unique = pimg_id_unique.tolist()
pimg_id_first_index = pimg_id_first_index.tolist()
pimg_id_counts = pimg_id_counts.tolist()

In [ ]:
home = pathlib.Path.home()

In [ ]:
for i, encoded_image_path in enumerate(images_to_be_indexed):

    
    new_path = indexed_dir.joinpath(f'frame_{i}.png')
    
    shutil.copy2(encoded_image_path, home.joinpath('temp_encoded.jpg'))
    
    !bash -c "~/bin/wavelet/jpeg /mnt/c/Users/sbiggs/temp_encoded.jpg /mnt/c/Users/sbiggs/temp_decoded.jpg"
    !bash -c "convert -depth 16 /mnt/c/Users/sbiggs/temp_decoded.jpg /mnt/c/Users/sbiggs/temp_decoded.png"
    
    shutil.copy2(home.joinpath('temp_decoded.png'), new_path)
    encoded_image_path.unlink()